In [1]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import os
import gc
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import transforms
from IPython.display import clear_output
import pandas as pd
import wandb
import random
import copy
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, CLIPProcessor, CLIPModel, AutoImageProcessor, AutoModel, AutoModelForCausalLM, AutoProcessor, VisionTextDualEncoderModel, VisionTextDualEncoderProcessor
from transformers import ViltProcessor, ViltModel





In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device1 = "cuda:1" if torch.cuda.device_count() > 1 else "cpu"

print(f"Thiết bị 0: {device}")
print(f"Thiết bị 1: {device1}")


Thiết bị 0: cuda:0
Thiết bị 1: cpu


In [3]:
!pip install einops

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


In [5]:
torch.cuda.empty_cache()


In [6]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import json
import torch

import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# Tải mô hình và tokenizer từ Hugging Face
model_name = "vinai/PhoGPT-4B-Chat"
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)  
config.init_device = "cuda"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Move the model to GPU
model.to("cuda")
model.eval()

PROMPT_TEMPLATE = "### Tạo thêm 1 câu mang sắc thái mỉa mai tương tự câu sau : {instruction}\n### Trả lời:"  
def generate_sarcastic_text(instruction, num_samples=5):
    input_prompt = PROMPT_TEMPLATE.format(instruction=instruction)
    input_ids = tokenizer(input_prompt, return_tensors="pt")['input_ids'].to("cuda")

    outputs = model.generate(
        input_ids=input_ids,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        max_new_tokens=100,
        num_return_sequences=num_samples,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    responses = [tokenizer.decode(output, skip_special_tokens=True).split("### Trả lời:")[1].strip() for output in outputs]
    return responses

# Thực hiện sinh văn bản cho các mẫu
# (Code phần sinh văn bản và lưu vào file JSON tương tự như trước)

print("Đã tải mô hình và sẵn sàng thực hiện dự đoán với cấu hình tối ưu cho GPU.")


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

attention.py:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

norm.py:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Đã tải mô hình và sẵn sàng thực hiện dự đoán với cấu hình tối ưu cho GPU.


In [7]:
torch.cuda.empty_cache()

In [8]:
import json
import torch
from tqdm import tqdm  # Import tqdm để hiển thị thanh tiến trình

input_file = "/kaggle/input/data-root/vimmsd-train.json"  # Đường dẫn tới tệp JSON đầu vào
output_file = "/kaggle/working/augmented_image_train_text_sarcasm_final_2.json"  # Đường dẫn tới tệp JSON đầu ra

import os

# Đường dẫn đến thư mục lưu ảnh
output_image_dir = "/kaggle/working/image_aug_for_text"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(output_image_dir, exist_ok=True)

print(f"Thư mục lưu ảnh đã được tạo tại: {output_image_dir}")

# Đường dẫn đến thư mục chứa ảnh
image_train_dir = '/kaggle/input/dsc-dataset/train-images/train-images' 
image_test_dir = '/kaggle/input/dsc-dataset/dev-images/dev-images'

# Lấy danh sách các tệp ảnh .jpg
image_train_paths = [os.path.join(image_train_dir, file) for file in os.listdir(image_train_dir) if file.endswith('.jpg')]
image_test_paths = [os.path.join(image_test_dir, file) for file in os.listdir(image_test_dir) if file.endswith('.jpg')]


with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

    
    

Thư mục lưu ảnh đã được tạo tại: /kaggle/working/image_aug_for_text


In [9]:
import re

def clean_sensitive_words(text):
    # Sử dụng regex để tìm các từ có dạng chữ + "." + chữ và loại bỏ dấu chấm
    cleaned_text = re.sub(r'(\w)\.(\w)', r'\1\2', text)
    return cleaned_text

In [10]:

# # Lọc tất cả các mẫu có nhãn "text-sarcasm"
# sample_data = {k: v for k, v in data.items() if v['label'] == 'text-sarcasm'}

# # Tạo dữ liệu mở rộng cho tất cả các mẫu "text-sarcasm"
# augmented_data = {}
# new_idx = len(sample_data)

# # Sử dụng tqdm để hiển thị tiến trình của vòng lặp
# for idx, item in tqdm(sample_data.items(), desc="Processing sarcasm data"):
#     augmented_data[idx] = item  # Giữ lại mẫu gốc

#     caption = item['caption']
#     caption = clean_sensitive_words(caption)
#     generated_captions = generate_sarcastic_text(caption)
    
#     # In câu gốc
#     print(f"Original caption [{idx}]: {caption}")

#     # Thêm 5 mẫu mới cho mỗi câu gốc và in từng câu sinh ra
#     for i, gen_caption in enumerate(generated_captions):
#         print(f"Generated caption {i+1} for [{idx}]: {gen_caption}")  # In caption đã sinh ra

#         augmented_data[f"{new_idx}_{i}"] = {
#             "image": item["image"],
#             "caption": gen_caption,
#             "label": item["label"]
#         }
#     new_idx += 1

#     # Làm mới bộ nhớ GPU sau khi sinh văn bản cho mỗi mẫu
#     torch.cuda.empty_cache()

# # Ghi dữ liệu đã mở rộng ra file JSON
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(augmented_data, f, ensure_ascii=False, indent=4)

# print(f"Dữ liệu mẫu 'text-sarcasm' đã được lưu vào {output_file}")


In [11]:
torch.cuda.empty_cache()

In [12]:
import imgaug.augmenters as iaa
import imgaug as ia
from PIL import Image
import numpy as np
import os
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Compatibility with NumPy changes
np.bool = np.bool_
np.complex = np.complex_

def augment(aug, image):
    return aug(image=image)

# Path to the image directory
image_path = "./output_images/text_sarcasm/"

# Create an augmentation condition for 50% probability
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define the augmentation pipeline
seq = iaa.Sequential(
    [
        iaa.Fliplr(0.5),  # Horizontally flip 50% of all images
        iaa.Flipud(0.2),  # Vertically flip 20% of all images
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  # Scale the image
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # Translate image
            rotate=(-45, 45),  # Rotate by -45 to +45 degrees
            shear=(-16, 16),  # Shear the image
            order=[0, 1],  # Nearest neighbor or bilinear interpolation
            cval=(0, 255),  # Fill in missing pixels (if any)
            mode=ia.ALL  # Apply all modes (from scikit-image)
        )),
        iaa.SomeOf((0, 5), [
            sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
            iaa.OneOf([
                iaa.GaussianBlur((0, 3.0)),
                iaa.AverageBlur(k=(2, 7)),
                iaa.MedianBlur(k=(3, 11))
            ]),
            iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
            iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
            iaa.SimplexNoiseAlpha(iaa.OneOf([
                iaa.EdgeDetect(alpha=(0.5, 1.0)),
                iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
            ])),
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
            iaa.OneOf([
                iaa.Dropout((0.01, 0.1), per_channel=0.5),
                iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
            ]),
            iaa.Invert(0.05, per_channel=True),
            iaa.Add((-10, 10), per_channel=0.5),
            iaa.AddToHueAndSaturation((-20, 20)),
            iaa.OneOf([
                iaa.Multiply((0.5, 1.5), per_channel=0.5),
                iaa.FrequencyNoiseAlpha(
                    exponent=(-4, 0),
                    first=iaa.Multiply((0.5, 1.5), per_channel=True),
                    second=iaa.ContrastNormalization((0.5, 2.0))
                )
            ]),
            iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5),
            iaa.Grayscale(alpha=(0.0, 1.0))
        ])
    ],
    random_order=True
)

Data_text_sc = {k: v for k, v in data.items() if v['label'] == 'text-sarcasm'}




/opt/conda/lib/python3.10/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `SimplexNoiseAlpha()` is deprecated. Use `BlendAlphaSimplexNoise` instead. SimplexNoiseAlpha is deprecated. Use BlendAlphaSimplexNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
/opt/conda/lib/python3.10/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/opt/conda/lib/python3.10/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn

In [13]:
import json

# Lọc các mẫu có nhãn 'text-sarcasm'
Data_text_sc = {k: v for k, v in data.items() if v['label'] == 'text-sarcasm'}

# Đường dẫn file JSON mà bạn muốn lưu
output_file = 'Data_text_sc.json'

# Ghi dữ liệu vào file JSON
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(Data_text_sc, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được lưu vào {output_file}")


Dữ liệu đã được lưu vào Data_text_sc.json


In [14]:
from sklearn.model_selection import train_test_split

# Đọc file JSON
with open('/kaggle/working/Data_text_sc.json', 'r', encoding='utf-8') as f:
    data_text_sc = json.load(f)

keys = list(data_text_sc.keys())
train_keys, val_keys = train_test_split(keys, test_size=0.2, random_state=42)





# Kiểm tra số lượng phần tử trong từng tập
print("Số lượng mẫu trong tập huấn luyện:", len(train_keys))
print("Số lượng mẫu trong tập kiểm tra:", len(val_keys))


Số lượng mẫu trong tập huấn luyện: 61
Số lượng mẫu trong tập kiểm tra: 16


In [15]:
# Tạo dữ liệu mở rộng cho các mẫu trong train_keys
augmented_data = {}
new_idx = len(train_keys)
output_image_dir = "/kaggle/working/augmented_images"
output_file = "/kaggle/working/augmented_data_text_sc.json"
os.makedirs(output_image_dir, exist_ok=True)


In [16]:
# Sử dụng tqdm để hiển thị tiến trình của vòng lặp
for idx in tqdm(train_keys, desc="Processing sarcasm data"):
    item = data_text_sc[idx]
    augmented_data[idx] = item  # Giữ lại mẫu gốc

    # Xử lý và in câu gốc
    caption = item['caption']
    caption = clean_sensitive_words(caption)
    generated_captions = generate_sarcastic_text(caption)
    
    print(f"Original caption [{idx}]: {caption}")

    # Thêm 5 mẫu mới cho mỗi câu gốc và in từng câu sinh ra
    for i, gen_caption in enumerate(generated_captions):
        print(f"Generated caption {i+1} for [{idx}]: {gen_caption}")  # In caption đã sinh ra

        # Tải ảnh gốc
        image_path = f"/kaggle/input/dsc-dataset/train-images/train-images/{item['image']}"
        print(image_path)
        
        # Kiểm tra và xử lý ảnh
        image = np.array(Image.open(image_path).convert("RGB"))

        # Áp dụng phép biến đổi ngẫu nhiên cho mỗi mẫu văn bản mới
        augmented_image = seq(image=image)

        # Lưu ảnh đã tăng cường vào một đường dẫn mới
        new_image_path = f"{output_image_dir}/augmented_{new_idx}_{i}.jpg"
        Image.fromarray(augmented_image).save(new_image_path)
        image_name = os.path.basename(new_image_path)

        # Thêm mẫu mới vào augmented_data
        augmented_data[f"{new_idx}_{i}"] = {
            "image": image_name,
            "caption": gen_caption,
            "label": item["label"]
        }
    new_idx += 1
    torch.cuda.empty_cache()  # Làm mới bộ nhớ GPU

# Ghi dữ liệu đã mở rộng ra file JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu mẫu 'text-sarcasm' đã được lưu vào {output_file}")

Processing sarcasm data:   0%|          | 0/61 [00:00<?, ?it/s]/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/mem

Original caption [699]: Đợt mưa lũ ở tỉnh Sindh, Pakistan đã khiến nước dâng lên cao. Thế là tụi nhện leo hết lên cây tạm trú, để lại những cây kẹo bông gòn như thế này 🐧
Generated caption 1 for [699]: Đợt mưa lũ ở tỉnh Sindh, Pakistan đã khiến nước dâng lên cao. Các cây kẹo bông gòn như thế này để lại một cảnh tượng hài hước.
/kaggle/input/dsc-dataset/train-images/train-images/fd4c1fc97ab96e2fe2402de220e6434dd327d7d9dd8f2dabb6d11f444883eaf1.jpg
Generated caption 2 for [699]: Những đợt mưa lũ ở tỉnh Sindh, Pakistan đã khiến nhiều ngôi nhà ngập lụt, và nhiều con vật đã mắc kẹt trong dòng nước lũ cao. Nhưng điều đó vẫn chưa kinh khủng bằng những cây kẹo bông gòn bị lũ cuốn trôi.
/kaggle/input/dsc-dataset/train-images/train-images/fd4c1fc97ab96e2fe2402de220e6434dd327d7d9dd8f2dabb6d11f444883eaf1.jpg
Generated caption 3 for [699]: Nước dâng cao khiến cây cối ngập lụt. Các loài vật đã leo lên cây để trú ẩn. Chúng để lại một cảnh tượng tan hoang sau đó!
/kaggle/input/dsc-dataset/train-images/

Processing sarcasm data:   3%|▎         | 2/61 [00:49<22:25, 22.80s/it]

Original caption [521]: Lưỡi không xương mà lại
Generated caption 1 for [521]: thật dài, nhưng lại sở hữu lắm xương.
/kaggle/input/dsc-dataset/train-images/train-images/f5bac19649b74b491a654d0c9fc0c9255df83dbdb8ea6b20634d750745a0b96f.jpg
Generated caption 2 for [521]: Không xương nên dễ gãy chứ sao !

Câu trên là một câu đố vui trong trò chơi dân gian Việt Nam, không phải là câu tục ngữ chính thức.
/kaggle/input/dsc-dataset/train-images/train-images/f5bac19649b74b491a654d0c9fc0c9255df83dbdb8ea6b20634d750745a0b96f.jpg
Generated caption 3 for [521]: ngọng.
/kaggle/input/dsc-dataset/train-images/train-images/f5bac19649b74b491a654d0c9fc0c9255df83dbdb8ea6b20634d750745a0b96f.jpg
Generated caption 4 for [521]: Lưỡi cũng chẳng xương mà uốn được.
/kaggle/input/dsc-dataset/train-images/train-images/f5bac19649b74b491a654d0c9fc0c9255df83dbdb8ea6b20634d750745a0b96f.jpg
Generated caption 5 for [521]: nói nhiều lời vô ích.
/kaggle/input/dsc-dataset/train-images/train-images/f5bac19649b74b491a654d0c9f

Processing sarcasm data:   5%|▍         | 3/61 [02:28<55:40, 57.59s/it]

...

Generated caption 3 for [9644]: Mặc dù bị đối thủ liên tục gỡ hòa, nhưng Arsenal vẫn còn cơ hội vô địch khi khoảng cách chỉ là 4 điểm và giải đấu vẫn còn nhiều trận đấu.
/kaggle/input/dsc-dataset/train-images/train-images/2e1abacc8afd3cb6899668b69ec9432e02aee080f5a1308ee9a52b5e5d1312a8.jpg
Generated caption 4 for [9644]: Mặc dù bị chấn thương và thể lực không đạt, nhưng các cầu thủ Arsenal vẫn chơi rất tốt và giành chiến thắng trước Chelsea.
/kaggle/input/dsc-dataset/train-images/train-images/2e1abacc8afd3cb6899668b69ec9432e02aee080f5a1308ee9a52b5e5d1312a8.jpg
Generated caption 5 for [9644]: "Mặc dù trận đấu giữa Arsenal và Tottenham Hotspur đã kết thúc với kết quả hòa, nhưng đội chủ nhà vẫn còn rất nhiều cơ hội chiến thắng nếu như chúng ta có thể tập trung vào mục tiêu chính của đội bóng, đó là vị trí trong top 4 Premier League."
/kaggle/input/dsc-dataset/train-images/train-images/2e1abacc8afd3cb6899668b69ec9432e02aee080f5a1308ee9a52b5e5d1312a8.jpg


Processing sarcasm data:  98%|█████████▊| 60/61 [1:02:47<00:33, 33.20s/it]

Original caption [957]: 🍎
Generated caption 1 for [957]: "Cuộc sống này chẳng thú vị như người ta vẫn tưởng."
/kaggle/input/dsc-dataset/train-images/train-images/0fd50138395071a32e59811aee77fcf0b6f4ad05c69256c8ed4cf7cf0d77f688.jpg
Generated caption 2 for [957]: "Mẹ ơi, tối qua con thấy mệt quá nên ngủ sớm. Con không ngờ hôm nay lại dậy muộn đến thế!"
/kaggle/input/dsc-dataset/train-images/train-images/0fd50138395071a32e59811aee77fcf0b6f4ad05c69256c8ed4cf7cf0d77f688.jpg
Generated caption 3 for [957]: "Tôi thấy một câu vô nghĩa như vậy mà lại được viết ra bởi một người có học thức".
/kaggle/input/dsc-dataset/train-images/train-images/0fd50138395071a32e59811aee77fcf0b6f4ad05c69256c8ed4cf7cf0d77f688.jpg
Generated caption 4 for [957]: "Mưa rơi lất phất như bụi trong không trung, nhưng chẳng có tia sét nào lóe lên cả."
/kaggle/input/dsc-dataset/train-images/train-images/0fd50138395071a32e59811aee77fcf0b6f4ad05c69256c8ed4cf7cf0d77f688.jpg
Generated caption 5 for [957]: "Cuộc sống thật đáng sợ

Processing sarcasm data: 100%|██████████| 61/61 [1:03:55<00:00, 62.88s/it]

Dữ liệu mẫu 'text-sarcasm' đã được lưu vào /kaggle/working/augmented_data_text_sc.json


In [17]:
torch.cuda.empty_cache()


In [18]:
# from tqdm import tqdm
# import torch
# import json
# import cv2  # Nếu cần sử dụng OpenCV để xử lý ảnh
# import torchvision.transforms as T
# from PIL import Image
# import random


# # Tạo dữ liệu mở rộng cho tất cả các mẫu "text-sarcasm"
# augmented_data = {}
# new_idx = len(sample_data)

# # Sử dụng tqdm để hiển thị tiến trình của vòng lặp
# for idx, item in tqdm(sample_data.items(), desc="Processing sarcasm data"):
#     augmented_data[idx] = item  # Giữ lại mẫu gốc

#     caption = item['caption']
#     caption = clean_sensitive_words(caption)
# #     generated_captions = generate_sarcastic_text(caption)
    
#     # In câu gốc
#     print(f"Original caption [{idx}]: {caption}")

#     # Thêm 5 mẫu mới cho mỗi câu gốc và in từng câu sinh ra
#     for i, gen_caption in enumerate(generated_captions):
#         print(f"Generated caption {i+1} for [{idx}]: {gen_caption}")  # In caption đã sinh ra

#         # Tải ảnh gốc
#         image_path = f"/kaggle/input/dsc-dataset/train-images/train-images/{item['image']}"

#         print(image_path)
#         image = np.array(Image.open(image_path).convert("RGB"))

#         # Áp dụng phép biến đổi ngẫu nhiên cho mỗi mẫu văn bản mới
# #         transform = transformations[i % len(transformations)]  # Lấy biến đổi từ danh sách
#         augmented_image = seq(image=image)

#         # Lưu ảnh đã tăng cường vào một đường dẫn mới
#         new_image_path = f"{output_image_dir}/augmented_{new_idx}_{i}.jpg"
#         Image.fromarray(augmented_image).save(new_image_path)
#         augmented_image.save(new_image_path)
#         image_name = os.path.basename(new_image_path)

#         # Thêm mẫu mới vào augmented_data
#         augmented_data[f"{new_idx}_{i}"] = {
#             "image": image_name,
#             "caption": caption,
#             "label": item["label"]
#         }
#         break
#     new_idx += 1
#     break
#     # Làm mới bộ nhớ GPU sau khi sinh văn bản cho mỗi mẫu
#     torch.cuda.empty_cache()

# # Ghi dữ liệu đã mở rộng ra file JSON
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(augmented_data, f, ensure_ascii=False, indent=4)

# print(f"Dữ liệu mẫu 'text-sarcasm' đã được lưu vào {output_file}")
